# The Presentation API

The Presentation API defines the IIIF data structure you are most likely to encounter. It is designed to be used by clients to display images to users; with different views of the image, related images, annotations, and other additional information displayed alongside the image. It could be used, for example, to display a book, containing a series of images of the pages, along with transcriptions of the text on each page, and annotations made by users.

The full specification is available at [IIIF Presentation API](https://iiif.io/api/presentation/3.0/).

In [ ]:
import json

## Structure 

IIIF manifests are made of a series of nested components in a hierarchical tree structure. Each component can generally have multiple children of the level below, with collections specifically being allowed to have child collections. The main components, using the example of a book to describe their contents, are:

- **Collection** - A collection of manifests, for example the collected works of an author.
- **Manifest** - A single book, containing pages and metadata.
- **Canvas** - A single page of the book, possibly containing multiple images or paragraphs of text.
- **Annotation page** - A page of annotations. 
- **Annotation** - A single annotation - information about a paragraph, or a region of an image.
- **Content** - The content of the annotation, rather than the descriptive metadata. An annotation might have multiple content items, for example a paragraph of text and a region of an image.

A diagram detailing how these layers and some additional components is available on the [Image API page](https://iiif.io/api/presentation/3.0/#21-defined-types).

## Building an example

The most effective way to learn a toolset is to use it, so here we are going to build a simple IIIF manifest from scratch by building a manifest for [this 1334 court roll](https://discovery.nationalarchives.gov.uk/details/r/C9060661) from the National Archives. This document is free to view, and can serve as a good example of a simple document to display. Note that, while the base URL we are going to use is live, the manifest we are going to create will contain many placeholders acting as examples of what you might expect to see - creating hypothetical URIs that are not live.

The process of building a manifest can run in either direction - either starting with collecting individual annotations onto their canvas, and working up to the manifest, or starting with the manifest and adding the details. The exact workflow for situations you may encounter will depend on the tools and data you have available. Here, we are going to start with the manifest and work down. 

Note about complexity: the manifest built here is going to be simple, with the required fields and a few extra details to work with the example document. Manifests can be built up into much more complex and rich documents, allowing the display of a large amount of content. 

### 1 - The top level

The top level of a manifest contains some standard information. The `@context` field gives information to the viewer about what sort of data to expect. `label` provides the title, with options to support multiple languages, and `behaviour` gives information on how the viewer should display the content. 

IIIF manifests are referenceable directly by their URI (meaning you can link to them directly). For a manifest, this URI is stored as the `id` field - putting this URI into a browser should return the manifest. For this example, we are imagining that the manifest made available by TNA by appending `/manifest.json` to the URL for the document on Discovery. 


In [ ]:
manifest = {
    "@context": "http://iiif.io/api/presentation/3/context.json",  # <-- telling the reader we're using IIIF, specifically version 3 of the Presentation API
    "id": "https://discovery.nationalarchives.gov.uk/details/r/C9060661/manifest.json", # <-- the URL for this manifest. For this example, we're using the National Archives' Discovery service, but appending a manifest.json
    "type": "Manifest", # <-- this is a manifest (as opposed to a collection or canvas, for example)
    "label": {
        "en": ["Petitioners: ? No Petitioner named Nature of Request: Extract of a court roll"] # <-- the document's title
    },
    "behavior": [
        "paged" # <-- providing information for the user agents on how to display the manifest
    ],
}

### 2 - A canvas for each page

Canvases are single views within a manifest, and while they typically are of a single image, they can contain multiple images or non-image content. As a result, there are few fields required by default - instead, fields become required depending on the content. The few requirements include that canvases must be referenceable by a URI, and that they are rectangular when described by their height and width. While not required, a title for each canvas is also strongly suggested. For this example, we are going to assume that each page of the court roll is supplied as a single image, thus a need for two canvases.

There are two fields available to store annotations on a canvas - `items` and `annotations`. The split is in place to allow the distinction of annotations that highlight a region of the image which stored in `annotations` (and require their motivation to be `painting`), and other annotations, such as transcriptions, stored in `items`. Annotations illustrate where fields can become required based on content - annotations containing OCR data must have the motivation `supplementing`. 

In [ ]:
page_0 = {
    "id": "https://discovery.nationalarchives.gov.uk/details/r/C9060661/canvas/p1", # <- imaginary URL, specifying that we want the information about the canvas for page 1
    "type": "Canvas",
    "label": {
        "en": ["Page 1"] 
    },
    "height": 1000, # <- in pixels, in this case just a placeholder
    "width": 800,
}

page_1 = {
    "id": "https://discovery.nationalarchives.gov.uk/details/r/C9060661/canvas/p2", 
    "type": "Canvas",
    "label": {
        "en": ["Page 2"] 
    },
    "height": 1000, 
    "width": 800,
}

### 3 - Annotations

Annotations follow the [Web Annotation Data Model](https://www.w3.org/TR/annotation-model/) - a widely used standard for storing and transferring information about resources. The within-IIIF requirements simply specify with some greater detail. For example, the annotations must also be referenceable by their URI (stored, as with the other components, in the `id` field), and contain a `motivation` field to describe what the annotation is for. IIIF annotations must also include the `target` field, which is a reference to the canvas or content the annotation is for. Other properties of the Web Annotation model are allowed, such as Fragment Selector, but are not required.

Annotations are collected together in an annotation page - allowing them to be stored separately, and embedded into multiple canvases as appropriate. These annotation pages are referenceable themselves with the URI in their `id` field, and store their annotations in the `items` field.

In [ ]:
page_1_annotation_page = {
    "id": "https://discovery.nationalarchives.gov.uk/details/r/C9060661/p1/annotations", 
    "type": "AnnotationPage",
}

page_1_annotation = {
    "id": "https://discovery.nationalarchives.gov.uk/details/r/C9060661/p2/annotations/1", 
    "type": "Annotation",
    "motivation": "painting", # <-- this annotation is an image (as opposed to, say, a comment)
    "body": {}, # <- this will be filled in in the next step
    "target": "https://discovery.nationalarchives.gov.uk/details/r/C9060661/canvas/p1", # <-- the URL of the canvas this annotation is associated with
}

page_2_annotation_page = {
    "id": "https://discovery.nationalarchives.gov.uk/details/r/C9060661/p2/annotations", 
    "type": "AnnotationPage",
}

page_2_annotation = {
    "id": "https://discovery.nationalarchives.gov.uk/details/r/C9060661/annotation/p2", 
    "type": "Annotation",
    "motivation": "painting",
    "body": {},
    "target": "https://discovery.nationalarchives.gov.uk/details/r/C9060661/canvas/p2",
}

#### 3.1 - Referencing external content

So far, it has been mentioned in a few places that the fields are referenceable by URI. This simply means that, should you request the URI (either via a browser or a command line), you should get that specific resource back (for eg, if you request the manifest by URI, you should receive the manifest, or if you only request a canvas, you should receive only the canvas). This opens a powerful feature of IIIF - the manifest does not need to contain all the information, only the references to it.

[Miiify](https://github.com/nationalarchives/miiify) takes advantage of this by storing annotations separately from the manifest, on a dedicated annotation server. Rather than writing the annotations directly to the manifest, you include a reference to the annotation server in the manifest. This allows you to store a large number of annotations without making the manifest unwieldy. When reading the manifest, the client reaches the ID - then, rather than reading further within the manifest for annotations, it sends a request to the server. The use of the W3C Web Annotation Data Model standard allows this process - the client can be confident that the annotations will be in a format it can understand.

### 4 - Content

Content is the final layer of the IIIF Presentation API, and is used to store the actual content of the annotation. This could be the image of the canvas, a transcription of text in the image, a translation, or more. The content should be referenceable by its URI (as always, in the `id` field), with the JSON describing the content stored in the `body` field of the parent annotation. The `type` field is required as well. Fields such as `format`, `height`, and `width` are not required, but are useful for the viewer to know how to display the content.

If the content is an image representing the canvas, then the `type` field should be set to `Image`. In this case, the `id` field should be set to the URI of the image - which, in this case, is allowed to be a IIIF image service, with the URI referring to a specific representation of the image. To see details of the IIIF Image API, see the [other notebook here](./image_api.ipynb).

In [ ]:
page_1_body = {
    "id": "https://discovery.nationalarchives.gov.uk/details/r/C9060661/1.jpg",
    "type": "Image",
    "format": "image/jpeg",
    "height": 1000,
    "width": 800,
    "service": [
        {
            "id": "https://discovery.nationalarchives.gov.uk/details/r/C9060661/1.jpg",
            "type": "ImageService3",
            "profile": "http://iiif.io/api/image/3/profiles/level2.json",
        }
    ]
}

page_2_body = {
    "id": "https://discovery.nationalarchives.gov.uk/details/r/C9060661/2.jpg",
    "type": "Image",
    "format": "image/jpeg",
    "height": 1000,
    "width": 800,
    "service": [
        {
            "id": "https://discovery.nationalarchives.gov.uk/details/r/C9060661/2.jpg",
            "type": "ImageService3",
            "profile": "http://iiif.io/api/image/3/profiles/level2.json",
        }
    ]
}

### 5 - Putting it all together

With each of the various levels of information collected, they can now be put together into a single manifest. Note how for most of these components, they are added as a list (despite being a single item) - this allows for multiples to be added, such as multiple canvases or annotations.

In [ ]:
page_1_annotation["body"] = page_1_body
page_2_annotation["body"] = page_2_body

page_1_annotation_page["items"] = [page_1_annotation]
page_2_annotation_page["items"] = [page_2_annotation]

page_0["items"] = [page_1_annotation_page]
page_1["items"] = [page_2_annotation_page]

manifest["items"] = [page_0, page_1]

print(json.dumps(manifest, indent=2))

### 6 - Referencing external content

So far, it has been mentioned in a few places that the fields are referenceable by URI (generally stored in the ID field). This means that, should you request the URI (either via a browser or a command line), you should get that specific resource back (for eg, if you request the manifest by URI, you should receive the manifest; if you only request a canvas, you should receive only the canvas). This opens a powerful feature of IIIF - the manifest does not need to contain all the information, only the URI referencing it.

[Miiify](https://github.com/nationalarchives/miiify) takes advantage of this by storing annotations separately from the manifest, on a dedicated annotation server. Rather than writing the annotations directly to the manifest, you include a reference to the annotation server in the manifest. This allows you to store a large number of annotations without making the manifest unwieldy. When reading the manifest, the client reaches the ID - then, rather than reading further within the manifest for annotations, it sends a request to the appropriate server. The use of the W3C Web Annotation Data Model standard allows this process as the client can be confident that the annotations will be in a format it expects and can understand.

In [ ]:
standard_annotation_page = {
    "id": "https://discovery.nationalarchives.gov.uk/details/r/C9060661/annotations", 
    "type": "AnnotationPage",
    "items": [
        # fully written out annotations
    ]
}

annotations_using_external_service = {
    "id": "https://example.org/miiify_annotations/C9060661", # On reaching this and seeing no 'items', the user agent would know to look for annotations at this URL. 
    "type": "AnnotationPage",
}

### Other fields

This notebook covers the basics of the IIIF Presentation API, and builds a manifest marginally more complex than the [simplest possible manifest example](https://iiif.io/api/cookbook/recipe/0001-mvm-image/). There are a variety of other optional fields that can be used to add more detail, eventually being used by viewers to provide more content or information to the user. Details of this additional information can be found in the [IIIF Presentation API 3.0 specification](https://iiif.io/api/presentation/3.0/), and a wide range of example manifests with more detail are available in the [IIIF cookbook](https://iiif.io/api/cookbook/). The documentation covers a wide range of use cases, and as a result can be complex, but is written to be as approachable as possible.